# <center>Generating Music Using LSTM Cells</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.

The idea is to read in MIDI files and convert them to arrays of notes. Then an RNN will be trained to predict the next note. Finally, music is generated by feeding a random string of notes to the RNN and having it iteratively predict the next note to form a song one note at a time.

### Things to test tomorrow:

* Add batch normalization - Loss won't go below ~3
* Add an extra layer (RNN and dense) - Meh
* Normalize between -1 to 1 - Didn't do much
* Decrease step size - MUCH better results with step size of 1
* Mess with batch size - Smaller seems better
* Mash-up 2 songs? - Works surprisingly well
* Converges? - No, different inputs should produce different tracks using the same model
* Try transfer learning and training the last few layers on this data?


### Visualizations
Cross correlations

In [53]:
# Imports
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
from music21 import converter, instrument, note, chord, stream, duration
from keras.models import Sequential
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM, LSTM, Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, History

## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [25]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append([str(element.pitch), element.offset, element.duration])
            elif isinstance(element, chord.Chord):
                song_note = '.'.join(str(n) for n in element.normalOrder)
                song.append([song_note, element.offset, element.duration])
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
#         Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append([str(element.pitch), element.offset, element.duration])
                elif isinstance(element, chord.Chord):
                    song_note = '.'.join(str(n) for n in element.normalOrder)
                    song.append([song_note, element.offset, element.duration])
            notes.append(song)

    return notes

# def get_notes_with_key(path, filter_key, mode):
#     """
#         Gets all notes and chords from midi files where the key matches the string input

#         Parameters
#         ----------
#         path : str
#             The path to the file
#         filter_key : str
#             The string to filter the key on
#         mode : int
#             The type of key used where:
#                 0 - key
#                 1 - major/minor
#                 else - key and major/minor
#     """
#     notes = []

#     for file in glob.glob(path + "*.mid"):        
#         song = []
#         midi = converter.parse(file)
        
#         # Only use music of the same key
#         key = midi.analyze('key')
#         if(mode==0):
#             key_string = str(key.tonic.name)
#         elif(mode==1):
#             key_string = str(key.mode)
#         else:
#             key_string = str(key.tonic.name + key.mode)
            
#         if(key_string==filter_key):
#             print("Parsing %s" % file)
#             notes_to_parse = None

#             try: # file has instrument parts
#                 s2 = instrument.partitionByInstrument(midi)
#                 notes_to_parse = s2.parts[0].recurse() 
#             except: # file has notes in a flat structure
#                 notes_to_parse = midi.flat.notes

#             for element in notes_to_parse:
#                 if isinstance(element, note.Note):
#                     song.append(str(element.pitch))
#                 elif isinstance(element, chord.Chord):
#                     song.append('.'.join(str(n) for n in element.normalOrder))

#     return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 1
input_dir_names = ["test", "Pokemon", "LoZ OOT", "Pokemon GSC", "Pokemon Route", "Undertale", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
notes = get_notes_with_key(input_path, "Cmajor", 2)
# notes = get_notes(input_path)

Parsing ../Pokemon MIDIs\Pokemon - Lavender town.mid
Parsing ../Pokemon MIDIs\Pokemon - Pkmn Elite 4.mid
Parsing ../Pokemon MIDIs\Pokemon - Pokemon Center.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Surfing.mid
Parsing ../Pokemon MIDIs\Pokemon Channel - Pokepad.mid
Parsing ../Pokemon MIDIs\Pokemon Colosseum - Outskirt Stand.mid
Parsing ../Pokemon MIDIs\Pokemon Colosseum - Relic Forest.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - DialgaPalkia Appear.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Hearthome City.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 202.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Route 205.mid
Parsing ../Pokemon MIDIs\Pokemon DiamondPearlPlatinum - Twinleaf Town Day.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Ecruteak CityCianwood City.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Johto Wild Pokemon Battle.mid
Parsing ../Pokemon MIDIs\Pokemon GoldSilverCrystal - Lavender 

I will now use an algo to determine the key of each song

In [3]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

key_count = print_key(input_path)
# key_count = print_key("../Undertale MIDIs/")
key_count

Parsing ../Pokemon Route MIDIs\Pokemon BlackWhite - Route 10.mid
C minor
Parsing ../Pokemon Route MIDIs\Pokemon BlackWhite - Route 3.mid
G major
Parsing ../Pokemon Route MIDIs\Pokemon BlackWhite - Route 4.mid
E major
Parsing ../Pokemon Route MIDIs\Pokemon BlackWhite - Route 6.mid
F minor
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 202.mid
C major
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 205 Night.mid
G major
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 205.mid
C major
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 209.mid
A major
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 225 Night.mid
D major
Parsing ../Pokemon Route MIDIs\Pokemon DiamondPearlPlatinum - Route 228 Night.mid
C minor
Parsing ../Pokemon Route MIDIs\Pokemon GoldSilverCrystal - Route 27.mid
A minor
Parsing ../Pokemon Route MIDIs\Pokemon GoldSilverCrystal - Route 29.mid
C major
Parsing ../Pokemon Route MIDI

{'Cminor': 2,
 'Gmajor': 3,
 'Emajor': 3,
 'Fminor': 1,
 'Cmajor': 3,
 'Amajor': 3,
 'Dmajor': 4,
 'Aminor': 1,
 'Bmajor': 1,
 'Fmajor': 1,
 'Dminor': 1,
 'A-major': 1,
 'F#major': 1,
 'B-major': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [26]:
possibleNotes = set([item[0] for sublist in notes for item in sublist])

# Processing for offsets
possibleOffsets = []
possibleDurations = []

# For each song
for index, song in enumerate(notes):
    song_length = len(song)
    
    # For each note, calculate the difference in offset between this and the previous note
    song_offsets = []
    song_durations = []
    for idx in range(song_length):
        offset = offset = round(song[idx][1] - song[idx - 1][1], 3) if idx > 1 else 0.0
        song_offsets.append(offset)
        if offset not in possibleOffsets:
            possibleOffsets.append(offset)
        
        duration = song[idx][2].quarterLength
        song_durations.append(duration)
        if duration not in possibleDurations:
            possibleDurations.append(duration)
            
    # Update the notes to reflect this
    for idx in range(song_length):
        notes[index][idx][1] = song_offsets[idx]
        notes[index][idx][2] = song_durations[idx]

n_notes = len(possibleNotes)
n_offset = len(possibleOffsets)
n_duration = len(possibleDurations)


possibleNotes = np.array(list(possibleNotes))
possibleOffsets = np.array(list(possibleOffsets))
possibleDurations = np.array(list(possibleDurations))
notes = np.array([list([list(subsublist) for subsublist in sublist]) for sublist in notes])
len(possibleNotes), len(possibleOffsets), len(possibleDurations)

(248, 25, 35)

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [27]:
def prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations):
    """ Prepare the sequences used by the Neural Network """
    song_end_indices = []
    sequence_length = 100
    step_size = 1

    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    offset_to_int = dict((offset, number) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    duration_to_int = dict((duration, number) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)

    network_input = []
    network_output_notes = []
    network_output_offset = []
    network_output_duration = []


    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([np.array([note_to_int[row[0]] / float(n_notes), offset_to_int[row[1]] / float(n_offset), duration_to_int[row[2]] / float(n_duration)]) for row in sequence_in])
            network_output_notes.append(np.array([note_to_int[sequence_out[0]]]))
            network_output_offset.append(np.array([offset_to_int[sequence_out[1]]]))
            network_output_duration.append(np.array([duration_to_int[sequence_out[2]]]))
        song_end_indices.append(len(network_input)-1)


    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 3))

    # Make one-hot-encoding
    network_output_notes = np_utils.to_categorical(network_output_notes, num_classes=n_notes)
    network_output_offset = np_utils.to_categorical(network_output_offset, num_classes=n_offset)
    network_output_duration = np_utils.to_categorical(network_output_duration, num_classes=n_duration)


    return (network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices)

network_input, network_output_notes, network_output_offset, network_output_duration, song_end_indices = prepare_sequences(notes, possibleNotes, possibleOffsets, possibleDurations)
network_input.shape

(13637, 100, 3)

## Constructing the model

I will now construct the model using CuDNNLSTM cells because they are significantly faster than regular LSTM cells due to being optimized for CuDA. I will have two CuDNNLSTM layers, followed by two dense layers and a final softmax activation layer to output the most probable result.

Hyperparameters:
* Optimizer - ADAM because it is considered one of the best
* Loss - categorical_crossentropy because it penalizes wrong predictions of multi-class problems best
* Epochs - More epochs are generally better as long as they don't overfit. I track loss over time and have checkpoints every 5 epochs so this will not be a problem
* Batch Size - This determines how many instances should be considered in each batch. Realistically, each different song would interfere with the other so I will reduce this.
    * Smaller Batch Size seems to add more variation

In [28]:
from keras import Input
from keras.models import Model

def create_network(network_input, n_notes, n_offset, n_duration):
    """ create the structure of the neural network """
    input = Input(shape=(network_input.shape[1], network_input.shape[2]))
    lstm_1 = CuDNNLSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True)(input)
    dropout_1 = Dropout(rate=0.3)(lstm_1)
    lstm_2 = Bidirectional(CuDNNLSTM(256, return_sequences=True))(dropout_1)
    dropout_2 = Dropout(rate=0.3)(lstm_2)
    lstm_3 = Bidirectional(CuDNNLSTM(256))(dropout_2)
    dense_1 = Dense(128)(lstm_3)
    dropout_3 = Dropout(rate=0.3)(dense_1)
    dense_2 = Dense(3, input_shape=(n_notes, n_offset, n_duration))(dropout_3)
    output_notes = Dense(n_notes, activation='softmax')(dense_2)
    output_offset = Dense(n_offset, activation='softmax')(dense_2)
    output_duration = Dense(n_duration, activation='softmax')(dense_2)
    
    model = Model(inputs=input, outputs=[output_notes, output_offset, output_duration])
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], optimizer='adam', loss_weights=[1., 1., 1.])
    return model

# Set up the model
model = create_network(network_input, n_notes, n_offset, n_duration)
history = History()

# Save on each epoch (because training isn't cheap!!!) and can use this to generate music for each checkpoint
outputDest = '../output/LSTM_' + input_dir_names[input_dir_choice] + '_' + str(int(time.time())) + '/'
if not os.path.exists(outputDest):
    os.makedirs(outputDest)

cp_callback = ModelCheckpoint(filepath=outputDest + "LSTMmodel_weights_{epoch:02d}.hdf5",
                              save_weights_only=True,
                              verbose=1,
                              period=5)

# Set parameters
n_epochs = 80
batch_size = 128
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100, 3)       0                                            
__________________________________________________________________________________________________
cu_dnnlstm_10 (CuDNNLSTM)       (None, 100, 256)     267264      input_4[0][0]                    
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 100, 256)     0           cu_dnnlstm_10[0][0]              
__________________________________________________________________________________________________
bidirectional_7 (Bidirectional) (None, 100, 512)     1052672     dropout_10[0][0]                 
____________________________________________________________________________________________

## Training the Model

I will save the final model, but keep checkpoints along the way to avoid overfitting and also use these to generate different midis.

In [35]:
model.fit(network_input, [network_output_notes, network_output_offset, network_output_duration], callbacks=[history, cp_callback], epochs=n_epochs, batch_size=batch_size)
model.save(outputDest + 'LSTMmodel_final.h5')

Epoch 1/80
13637/13637 [==============================] - 11s 842us/step - loss: 4.0460 - dense_18_loss: 2.9288 - dense_19_loss: 0.2132 - dense_20_loss: 0.9039
Epoch 2/80
13637/13637 [==============================] - 11s 836us/step - loss: 4.1947 - dense_18_loss: 3.0012 - dense_19_loss: 0.2635 - dense_20_loss: 0.9300
Epoch 3/80
13637/13637 [==============================] - 11s 837us/step - loss: 4.2038 - dense_18_loss: 3.0107 - dense_19_loss: 0.2434 - dense_20_loss: 0.9497
Epoch 4/80
13637/13637 [==============================] - 11s 839us/step - loss: 4.0129 - dense_18_loss: 2.8983 - dense_19_loss: 0.1999 - dense_20_loss: 0.9147
Epoch 5/80
13637/13637 [==============================] - 11s 838us/step - loss: 3.9649 - dense_18_loss: 2.8577 - dense_19_loss: 0.1961 - dense_20_loss: 0.9111

Epoch 00005: saving model to ../output/LSTM_Pokemon_1571016778/LSTMmodel_weights_05.hdf5
Epoch 6/80
13637/13637 [==============================] - 11s 840us/step - loss: 3.9543 - dense_18_loss: 2.846

13637/13637 [==============================] - 12s 855us/step - loss: 3.6426 - dense_18_loss: 2.5450 - dense_19_loss: 0.1956 - dense_20_loss: 0.90207s - loss: 3.6957 - dense_18_loss: 2.5772 - dense_19_loss: 0.2041 - dense - ETA: 5s - loss: 3.6923 - dense_18_loss: 
Epoch 42/80
13637/13637 [==============================] - 12s 855us/step - loss: 3.6466 - dense_18_loss: 2.5400 - dense_19_loss: 0.2071 - dense_20_loss: 0.89954s - loss: 3.6560 - dense_18_loss: 2.5431 - dense_1
Epoch 43/80
13637/13637 [==============================] - 12s 857us/step - loss: 3.6387 - dense_18_loss: 2.5328 - dense_19_loss: 0.2054 - dense_20_loss: 0.9004
Epoch 44/80
13637/13637 [==============================] - 12s 865us/step - loss: 3.6115 - dense_18_loss: 2.5136 - dense_19_loss: 0.1958 - dense_20_loss: 0.9021
Epoch 45/80
13637/13637 [==============================] - 12s 870us/step - loss: 3.5957 - dense_18_loss: 2.5054 - dense_19_loss: 0.1964 - dense_20_loss: 0.89393s - loss: 3.5905 - dense_18_loss: 2.5121

In [36]:
# Plot the model losses
pd.DataFrame(history.history).plot()
plt.savefig(outputDest + 'LSTM_Loss_per_Epoch1.png', transparent=True)
plt.close()

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [11]:
# model.load_weights(outputDest + "LSTMmodel_weights_48.hdf5")

In [82]:
def generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices):
    """ Generate notes from the neural network based on a sequence of notes """
    # create a dictionary to map pitches to integers
    pitchnames = sorted(possibleNotes)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # create a dictionary to map offset to integers
    offsetnames = sorted(possibleOffsets)
    int_to_offset = dict((number, offset) for number, offset in enumerate(offsetnames))
    
    # create a dictionary to map duration to integers
    durationnames = sorted(possibleDurations)
    int_to_duration = dict((number, duration) for number, duration in enumerate(durationnames))
    
    # find number of each possible choice for normalization
    n_notes = len(possibleNotes)
    n_offset = len(possibleOffsets)
    n_duration = len(possibleDurations)
    
    # choose a random point to start
#     start = np.random.randint(0, len(song_end_indices)-1)
#     pattern = network_input[song_end_indices[start]]
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
#     np.random.shuffle(pattern)
    sequence_length = pattern.shape[0]
    n_dim = pattern.shape[1]
    
    prediction_output = []
    
    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, sequence_length, n_dim))
        prediction_input = prediction_input

        prediction = model.predict(prediction_input, verbose=0)
        
        note_int = np.argmax(prediction[0])
        note_normalized = note_int / float(n_notes)
        note = int_to_note[note_int]
        
        offset_int = np.argmax(prediction[1])
        offset_normalized = offset_int / float(n_offset)
        offset = int_to_offset[offset_int]
                
        duration_int = np.argmax(prediction[2])
        duration_normalized = duration_int / float(n_duration)
        duration = int_to_duration[duration_int]
        
        result = np.array([note_normalized, offset_normalized, duration_normalized])
        full_prediction = np.array([note, offset, duration])
        
        prediction_output.append(full_prediction)
        pattern = np.append(pattern, result)
        pattern = pattern[3:len(pattern)]
        
    print([str(x[0]) for x in prediction_output])
    
    return prediction_output

prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices)

['C3', 'E5', 'D3', 'E4', 'G3', 'C3', '0.4', 'F3', 'E4', 'G3', 'G3', 'F3', 'D3', 'C5', 'G3', 'A4', 'A4', '2.7', 'G3', 'E5', 'E4', 'F3', 'G3', 'D5', 'C4', '9', '11.2', 'E4', 'D5', '11.2', 'C4', 'C5', 'C4', 'C4', 'A4', 'C4', 'C4', 'C4', 'C4', 'F3', 'G3', 'E4', 'G3', 'C4', 'G3', 'E5', 'C5', 'E5', 'C4', 'C5', 'A4', 'C4', 'D5', 'G3', '2.7', 'E5', 'E5', 'F3', 'E5', 'E3', 'F3', '2.7', 'E5', 'E4', 'E5', 'D3', 'E5', 'F3', 'E5', 'G3', '2.7', '0.4', 'E5', '2.7', '0.4', 'G2', 'E5', 'C5', 'C4', '11.2', 'C4', 'E5', '0.4', '2.7', '0.4', '2.7', 'E5', 'C4', 'E5', 'C4', 'E5', 'G3', 'E5', 'G3', 'E5', '2.7', '2.7', 'E5', 'E4', 'F3', 'E5', 'C5', 'C4', 'G3', 'G3', '11.2', '0.5', 'E2', 'G3', 'A4', 'G3', '0.4', 'G3', '2.7', 'E2', '0.5', 'E5', 'E5', 'E4', 'C5', 'C4', '11.2', 'C3', '0.4', 'E5', 'C4', 'F3', 'E5', 'G3', '2.7', 'G2', '2.7', 'E5', 'A4', 'E4', 'E4', 'C4', 'F3', 'G3', 'G3', 'E4', 'E5', 'G3', 'E5', 'G3', 'C6', 'G3', '11.2', 'C3', '2.7', 'G3', 'E5', 'G3', 'E5', 'C4', 'E5', 'E4', 'F3', '0.4', '0.4', '0.4

Next, I will create a midi using these notes and save to a file

In [83]:
from music21 import duration as D

def create_midi(prediction_output, filename, instrument_choice):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []
    output_notes.append(instrument_choice)

    # create note and chord objects based on the values generated by the model
    count = 0
    for pattern in prediction_output:
        note_str = pattern[0]
        offset_str = pattern[1]
        duration_str = pattern[2]
        if "#-" in note_str:# To fix a rare exception using 2 accidentals
            continue
        # pattern is a chord
        if ('.' in note_str) or note_str.isdigit():
            notes_in_chord = note_str.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(note_str)
            new_note.offset = offset
            new_note.duration = D.Duration(float(duration_str))
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += (float(prediction_output[count + 1][1])) if (count + 1 < len(prediction_output)) else 0
        count += 1

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# Select instrument
instruments = {
    'piano': instrument.Piano(),
    'flute': instrument.Flute(),
    'clarinet': instrument.Clarinet(),
    'ocarina': instrument.Ocarina(),
    'harmonica': instrument.Harmonica(),
    'steel_drum': instrument.SteelDrum(),
    'vocals': instrument.Vocalist(),
    'soprano': instrument.Soprano(),
    'guitar': instrument.Guitar(),
    'elec_guitar': instrument.ElectricGuitar(),
    'violin': instrument.Violin(),
    'saxophone': instrument.Saxophone(),
    'trombone': instrument.Trombone(),
    'trumpet': instrument.Trumpet(),
    'english_horn': instrument.EnglishHorn(),
}
instrument_string = 'ocarina'
instrument_choice = instruments[instrument_string]

create_midi(prediction_output, outputDest + 'LSTM_output_' + instrument_string + '_X', instrument_choice)
# create_midi(prediction_output, 'lavender')

Or a batch production could be performed of the final model

In [38]:
album_length = 10
for count_output in range(album_length):
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations, song_end_indices)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + instrument_string + '_'+ str(count_output), instrument_choice)
    print(f"Created at {outputDest + 'LSTM_output_final' + str(count_output)}")

['B3', 'D3', 'G3', 'G3', 'C4', '7', 'E5', 'C4', '7', 'C6', '0', 'E5', '0.4', '0', '7', '0', '11.2', '7', '2.7', '11.2', '0', 'E5', '2.7', 'E5', 'G2', 'C3', '0.4', '7.0', '8.11', 'E5', '7', '0', '2.7', 'E5', 'C4', '2.7', 'E5', 'E5', 'E5', 'E5', 'C6', 'C5', 'C5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', '0', '0', 'C6', '2.7', '2.7', '2.7', '0', '0', '7', '7', '2.7', 'E5', 'E5', 'F3', '2.7', '7', 'E5', 'E5', 'E5', '11.2', '2.7', '0', '0', '7', '7', 'E5', '0', 'E5', '11.2', 'E4', 'C4', 'G5', 'C5', 'G3', 'E5', 'F3', 'C5', '7', '11.2', '0', '0', 'E5', 'E5', '2.7', 'G3', 'C6', 'F3', 'C4', 'E5', 'E5', 'E5', 'E5', 'E5', 'C6', 'C6', '7', '7.0', '0', 'E5', '7', '0', 'E5', '0', 'E5', '0', 'G3', 'C6', '0', 'D5', 'C6', 'E5', 'E5', 'E5', '11.2', '7', '7', '7', '7', '7', 'C6', 'E5', '2.7', 'E5', 'E5', '7', '7', '7.0', '0', '0', '2.7', '0.4', '7', '7', 'E5', '0.4', 'C5', 'C5', '7.0', 'E5', '0', '0', '7', '7.0', '2.7', '0.4', '7', '7', '2.7', '2.7', 'E5', 'E5', '0', 'G3', 'E5', '7', '

Created at ../output/LSTM_Pokemon_1571016778/LSTM_output_final2
['E4', 'E4', '7.0', 'C4', '0.5', '0.5', '0.5', 'C3', '2.7', '0.4', '0.4', 'G3', '0.4', 'C5', '11.2', 'E4', '2.7', '2.7', '0.4', '2.7', 'C5', 'E4', 'E5', 'F3', '2.7', 'E5', 'E5', 'G3', 'F3', 'E2', 'C6', '11.2', 'F3', 'G3', 'E5', '11.2', 'G2', '11.2', 'E2', 'C6', '11.2', 'E5', 'C4', 'E5', 'E5', 'G3', '11.2', 'G2', 'G4', '0.5', 'E5', '11.2', 'G3', 'E5', 'E4', '2.7', '2.7', 'E5', 'E4', 'E5', 'G3', '2.7', 'G3', 'E4', 'C4', '11.2', 'C6', 'D5', 'C4', 'C6', 'G3', '0.4', 'G3', '0.4', 'C3', '0.4', 'C5', 'G3', '2.7', 'G2', '0.4', 'E2', 'E2', '2.7', 'C3', 'F3', '2.7', 'F3', 'G3', 'E5', 'C5', 'C5', 'C4', 'C4', 'C5', '0', '11.2', 'F3', 'C6', 'F3', '7.0', 'E2', '0.5', 'E5', 'E4', 'C6', 'C5', 'C4', 'E5', 'D5', 'G3', 'E5', 'G3', '7.0', '2.7', 'E5', 'G3', '2.7', '0.4', 'E2', 'C3', 'E5', 'F3', '2.7', 'C3', '7.0', 'E5', 'E4', 'C4', '0', 'E5', 'E4', 'C6', 'F3', '7', 'G3', '7.0', 'C4', 'E5', 'E4', '2.7', '11.2', 'G4', 'C4', 'G3', 'G5', 'E4', 'E

Created at ../output/LSTM_Pokemon_1571016778/LSTM_output_final5
['G3', 'B3', 'C4', '0', 'C5', 'G3', 'E5', 'C5', 'E4', 'G3', 'C4', '7', 'E5', 'E4', 'E4', 'C4', '0', 'C4', '7.0', 'G3', 'F3', '7', 'C5', 'G3', 'G#3', 'C5', '11.2', 'C5', 'D5', 'C4', 'A3', '7', 'E5', 'F3', '7', 'E5', 'D5', 'D3', 'F3', 'C5', 'E5', 'F3', 'F3', 'E5', 'E5', 'E3', 'F3', 'C5', 'G5', 'D3', 'F3', 'C5', 'C5', 'D3', 'G3', 'C5', 'D3', 'G3', 'C5', 'F3', 'G3', 'G4', 'C4', 'G3', 'E4', 'C5', 'E4', 'F3', 'G3', 'G3', 'G3', 'C4', 'E4', 'G4', 'C4', 'C5', 'F3', 'C5', 'G3', 'D5', 'D5', 'E5', 'C4', 'C3', 'F3', 'E4', 'F3', 'G4', 'G3', 'E4', 'A4', '11.2', 'C4', 'F3', 'G4', 'F3', 'G3', 'G4', 'G3', '0', 'C5', '2.7', 'C5', 'G3', 'E4', 'E5', 'E4', '0', 'G5', 'C3', 'F3', 'C5', 'C4', 'G3', '0', 'G3', 'E4', 'E5', 'F3', 'C4', 'G3', 'C4', '7', '2.7', 'G5', 'E4', 'E5', 'E5', 'G3', 'G3', 'G3', 'C4', 'G4', 'C5', 'G3', 'C5', 'C4', 'C5', 'E4', 'E4', 'C5', 'C4', 'G3', 'F3', 'E5', '11.2', 'E4', 'G4', 'C3', 'E5', 'G3', 'G3', 'G3', 'D3', 'E5', 'C4',

Created at ../output/LSTM_Pokemon_1571016778/LSTM_output_final8
['C6', 'C5', 'G4', 'E4', 'G4', 'G4', 'G4', 'G4', 'G4', 'C5', 'G4', 'C5', 'C5', 'E4', 'E5', 'G3', 'E5', 'G3', '2.7', 'E5', 'C5', '0', 'E5', '7.0', '2.7', '7', '7', '7', 'E5', '7.0', 'G5', 'D3', 'E5', '0', 'E5', '0.4', 'C6', '0.4', '7', '2.7', '7', '7', 'E5', '0.4', 'E5', 'C5', '2.7', '0.4', '2.7', '0', '2.7', '0.4', '7', '7', 'E5', '7.0', 'G5', 'C4', '7', 'E5', '7', '2.7', 'E5', '0', '7.0', '7', 'E5', 'C6', 'E5', 'C6', '2.7', 'G3', 'A4', 'G5', '0.4', '7', '2.7', 'C6', '0', '2.7', '0', 'G3', '0.5', '2.7', '7', 'C6', '2.7', 'F3', 'G5', 'C6', 'E5', '0', '7', 'C5', 'E5', '11.2', '7', 'C6', 'C6', 'E5', '0', 'E5', 'E5', '2.7', 'C5', 'E5', '0', 'C6', 'C6', 'C6', 'E2', 'C6', 'C6', '7', '2.7', '2.7', '2.7', '0.4', '0', '0', '7', '0', '7', '2.7', '0', '0', 'G3', '2.7', 'F3', 'C6', '7', '7', '7', 'C6', '2.7', '7.0', '0.4', '7', '4.5', '7', '7.0', '7.0', '2.7', '7.0', '0.4', 'F3', 'E5', '0.4', 'E5', '7.0', '0', '0', 'E5', 'F3', '7', 'C

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [23]:
# Have each model make a song
count = 0
filepaths = glob.glob(outputDest + "*.hdf5")
for model_path in filepaths:
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, network_input, possibleNotes, possibleOffsets, possibleDurations)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1